# LeetCode 10
![lc-10](./assets/question.jpg)
![lc-10](./assets/constraints.jpg)

> Observations:
> - The lengths of the input and patterns strings are at least 1 character long
> - Of course, only the pattern can carry '.' and '*' as characters, while the rest remains to be lowercase letters
> - For all '*', then i - 1 is a position with a valid character (either '.' or a lowercase letter)
> - Most importantly, "*" matches zero or more of the preceding element
>   - This means that strange patterns, such as "c*ab" can still match with an input string "ab" since "c" can simply be repeated zero times

![lc-10-ex1](./assets/ex1.jpg)
![lc-10-ex2](./assets/ex2.jpg)
![lc-10-ex3](./assets/ex3.jpg)

> Notes:
> - For example 1, the pattern "a" does not match that of the input string "aa" for relatively trivial reasons
>   - However, note that the index goes out of bounds for the pattern before the string since the pattern itself is shorter than the input string
> - For example 2, the pattern "a*" does match that of the input string "aa" because the "a*" can repeat itself once (repeat the "a" once) to match that of "aa"
>   - Unlike example 1, we cannot traverse the strings using a common index, this may lead to the necessity for two pointers, one for the index of the input and the other for the pattern
>   - To move to the next index, it's trivial to increment the respective pointer by 1; however, if there is a star, then the pattern pointer has to increment by 2
> - For example 3, the pattern ".*" does match with "ab" because "a" matches with "." and, since there is a star, then the "." can repeat to match up with "b" 
> - Perhaps a recursive solution will be particularly helpful in solving such a problem intuitively considering our irregular methods for incrementations
>   - To settle on some base cases, one important one is that if pointer_input and pointer_pattern are both out of bounds, then it must be true that they match
>   - If pointer_pattern is out of bounds and pointer_input isn't, then it must be the case that the pattern does not match the input
>   - A match can be defined as when s[pointer_input] == p[pointer_pattern] or p[pointer_pattern] == '.'
>       - An edge case to this is that pointer_input < len(s) must also be true, otherwise the match is false 
>       - Edge case derived from a example s = "aa" and p = "aaa", where pointer_input would be outside the bounds of the input string s yet there are still remainds to the pattern p
> - If there is a character match and pointer_pattern + 1 == "*", then we should return a call to check the next element at pointer_input and remaind at pointer_pattern to respect the star
> - If there is no character match and pointer_pattern + 1 == "*", then we should return a call to check the next element at pointer_pattern by doing pointer_pattern + 2 and remaining at pointer_input
> - And, if there is no "*" and there is a match, then trivially, we just increment both pointers by 1 
> - Otherwise, we return false to indicate that the two strings do not match

> Additional Notes:
> - Note that we always have two decisions when met with a star, either repeat the letter or skip the letter
> - To add, we have two decisions for each character of the input string, and results in a O(2^n) time complexity with the current algorithm
> - Since it is a dynamic programming problem, we can implement a cache to store intermediate results bringing it to an overall O(m * n) time complexity where m and n are the lengths of input and pattern

> ### Algorithm
> - We need a cache to store our intermediate results, "cache"
> - We need a recursive function called "traverse_strings" which will pass pointers, "pointer_input" and "pointer_pattern" to keep track of the current index of each given string
> - Base cases:
>   - if (pointer_input, pointer_pattern) is in cache, then return cache[(pointer_input, pointer_pattern)]
>   - if pointer_input >= len(s) and pointer_pattern >= len(p) then return True (since if they both become out of bounds at the same time, then every character must have passed)
>   - if pointer_pattern >= len(p) then return False (since if pointer_pattern reaches out of bounds faster than the pointer_input, it must be the case the pattern does not match the string)
> - We must check if the characters at pointer_input and pointer_pattern are the same
>   - is_match = pointer_input < len(s) and (s[pointer_input] == p[pointer_pattern] or p[pointer_pattern] == '.') 
> - If pointer_pattern + 1 is not out of bounds and pointer_pattern + 1 is a "*" then...
>   - if is_match then cache[(pointer_input, pointer_pattern)] = traverse_strings(pointer_input + 1, pointer_pattern) and return cache[(pointer_input, pointer_pattern)] 
>       - (if there is a match, then we don't increment the repeatable character since we don't know about it's usefulness for the future, so we just increment the pointer_input)
>   - if not is_match then cache[(pointer_input, pointer_pattern)] = traverse_strings(pointer_input, pointer_pattern + 2) return cache[(pointer_input, pointer_pattern)] 
>       - (so as to skip the "*" character and check if the rest of the pattern suffices as a match)
> - If there is a match, then cache[(pointer_input, pointer_pattern)] = traverse_strings(pointer_input + 1, pointer_pattern + 1) and return cache[(pointer_input, pointer_pattern)]
> - Otherwise, it must not match and cache[(pointer_input, pointer_pattern)] = False and return False as a result

## Implementation

In [3]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        cache = {}
        def traverse_strings(ptr_input, ptr_pattern):
            if ((ptr_input, ptr_pattern) in cache):
                return cache[(ptr_input, ptr_pattern)]
            if ((ptr_input >= len(s)) and (ptr_pattern >= len(p))):
                return True
            if (ptr_pattern >= len(p)):
                return False
            is_match = ((ptr_input < len(s)) and (s[ptr_input] == p[ptr_pattern] or p[ptr_pattern] == '.'))
            if ((ptr_pattern + 1 < len(p)) and (p[ptr_pattern + 1] == '*')):
                    cache[(ptr_input, ptr_pattern)] = (traverse_strings(ptr_input, ptr_pattern + 2) or (is_match and traverse_strings(ptr_input + 1, ptr_pattern)))
                    return cache[(ptr_input, ptr_pattern)]
            if (is_match):
                cache[(ptr_input, ptr_pattern)] = traverse_strings(ptr_input + 1, ptr_pattern + 1)
                return cache[(ptr_input, ptr_pattern)]
            cache[(ptr_input, ptr_pattern)] = False
            return False
        return traverse_strings(0, 0)

In [4]:
sol = Solution()
print('Ex 1:')
print(' Result:', sol.isMatch('aa', 'a'))
print(' Desire: False')
print('Ex 2:')
print(' Result:', sol.isMatch('aa', 'a*'))
print(' Desire: True')
print('Ex 3:')
print(' Result:', sol.isMatch('ab', '.*'))
print(' Desire: True')

Ex 1:
 Result: False
 Desire: False
Ex 2:
 Result: True
 Desire: True
Ex 3:
 Result: True
 Desire: True


> Extra notes:
> - Completely overlooked, the original algorithm was implemented in such a way that failed to pass tests such as input = "aa" and pattern = "a*a"
> - So, for where we check for the "*", the line after no longer checks whether there is a match and instead goes both routes for when a letter is repeated and a letter is not. Since there is an or in between, then if it ever resolves true, then that will be the result.

> ### Final Verdict
> - Since we cache our results and retrieve them in O(1) time, then traversing to check if the two strings match can be done in O(m * n) time where m and n are the lengths of the input and pattern strings, respectively. In terms of our cache, since we utilise at most O(m * n) key-value pairs, then the space complexity of this algorithm is O(m * n) as well